In [129]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from scipy.linalg import block_diag
from __future__ import division

In [130]:
class LaneTracker:
    def __init__(self, n_lanes, proc_noise_scale, meas_noise_scale, process_cov_parallel=0, proc_noise_type='white'):
        self.n_lanes = n_lanes
        self.meas_size = 4 * self.n_lanes
        self.state_size = self.meas_size * 2
        self.contr_size = 0

        self.kf = cv2.KalmanFilter(self.state_size, self.meas_size, self.contr_size)
        self.kf.transitionMatrix = np.eye(self.state_size, dtype=np.float32)
        self.kf.measurementMatrix = np.zeros((self.meas_size, self.state_size), np.float32)
        for i in range(self.meas_size):
            self.kf.measurementMatrix[i, i*2] = 1

        if proc_noise_type == 'white':
            block = np.matrix([[0.25, 0.5],
                               [0.5, 1.]], dtype=np.float32)
            self.kf.processNoiseCov = block_diag(*([block] * self.meas_size)) * proc_noise_scale
        if proc_noise_type == 'identity':
            self.kf.processNoiseCov = np.eye(self.state_size, dtype=np.float32) * proc_noise_scale
        for i in range(0, self.meas_size, 2):
            for j in range(1, self.n_lanes):
                self.kf.processNoiseCov[i, i+(j*8)] = process_cov_parallel
                self.kf.processNoiseCov[i+(j*8), i] = process_cov_parallel

        self.kf.measurementNoiseCov = np.eye(self.meas_size, dtype=np.float32) * meas_noise_scale

        self.kf.errorCovPre = np.eye(self.state_size)

        self.meas = np.zeros((self.meas_size, 1), np.float32)
        self.state = np.zeros((self.state_size, 1), np.float32)

        self.first_detected = False

    def _update_dt(self, dt):
        for i in range(0, self.state_size, 2):
            self.kf.transitionMatrix[i, i+1] = dt

    def _first_detect(self, lanes):
        for l, i in zip(lanes, range(0, self.state_size, 8)):
            self.state[i:i+8:2, 0] = l
        self.kf.statePost = self.state
        self.first_detected = True

    def update(self, lanes):
        if self.first_detected:
            for l, i in zip(lanes, range(0, self.meas_size, 4)):
                if l is not None:
                    self.meas[i:i+4, 0] = l
            self.kf.correct(self.meas)
        else:
            if lanes.count(None) == 0:
                self._first_detect(lanes)

    def predict(self, dt):
        if self.first_detected:
            self._update_dt(dt)
            state = self.kf.predict()
            lanes = []
            for i in range(0, len(state), 8):
                lanes.append((state[i], state[i+2], state[i+4], state[i+6]))
                print(lanes)
            return lanes
        else:
            return None

In [131]:
def hsv_filter(image, min_val_y, max_val_y,  min_val_w, max_val_w):
    """
    A function returning a mask for pixels within min_val - max_val range
    Inputs:
    - image - a BGR image you want to apply function on
    - min_val_y - array of shape (3,) giving minumum HSV values for yellow color
    - max_val_y - array of shape (3,) giving maximum HSV values for yellow color
    - min_val_w - array of shape (3,) giving minumum HSV values for white color
    - max_val_w - array of shape (3,) giving maximum HSV values for white color
    Returns:
    - img_filtered - image of pixels being in given threshold
    
 gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
 mask = cv2.inRange(frame,lower_color_bounds,upper_color_bounds )
 mask_rgb = cv2.cvtColor(mask,cv2.COLOR_GRAY2BGR)
 frame = frame & mask_rgb

    """
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask_yellow = cv2.inRange(hsv, min_val_y, max_val_y)
    mask_white = cv2.inRange(hsv, min_val_w, max_val_w)
    mask = cv2.bitwise_or(mask_yellow, mask_white)
    img_filtered = cv2.bitwise_and(image, image, mask=mask)
    
    return img_filtered

In [132]:
class LaneDetector:
    def __init__(self, road_horizon, prob_hough=True):
        self.prob_hough = prob_hough
        self.vote = 50
        self.roi_theta = 0.59
        self.road_horizon = road_horizon

    def _standard_hough(self, img, init_vote):
        # Hough transform wrapper to return a list of points like PHough does
        lines = cv2.HoughLines(img, 1, np.pi/180, init_vote)
        points = [[]]
        for l in lines:
            for rho, theta in l:
                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a*rho
                y0 = b*rho
                x1 = int(x0 + 1000*(-b))
                y1 = int(y0 + 1000*a)
                x2 = int(x0 - 1000*(-b))
                y2 = int(y0 - 1000*a)
                points[0].append((x1, y1, x2, y2))
                print(points)
        return points

    def _base_distance(self, x1, y1, x2, y2, width):
        # compute the point where the give line crosses the base of the frame
        # return distance of that point from center of the frame
        if x2 == x1:
            return (width*0.5) - x1
        m = (y2-y1)/(x2-x1)
        c = y1 - m*x1
        base_cross = -c/m
        return (width*0.5) - base_cross

    def _scale_line(self, x1, y1, x2, y2, frame_height):
        # scale the farthest point of the segment to be on the drawing horizon
        if x1 == x2:
            if y1 < y2:
                y1 = self.road_horizon
                y2 = frame_height
                return x1, y1, x2, y2
            else:
                y2 = self.road_horizon
                y1 = frame_height
                return x1, y1, x2, y2
        if y1 < y2:
            m = (y1-y2)/(x1-x2)
            x1 = ((self.road_horizon-y1)/m) + x1
            y1 = self.road_horizon
            x2 = ((frame_height-y2)/m) + x2
            y2 = frame_height
        else:
            m = (y2-y1)/(x2-x1)
            x2 = ((self.road_horizon-y2)/m) + x2
            y2 = self.road_horizon
            x1 = ((frame_height-y1)/m) + x1
            y1 = frame_height
        return x1, y1, x2, y2

    def detect(self, frame):
        
        img= frame
        
        min_val_y = np.array([15,80,190])
        max_val_y = np.array([30,255,255])
        min_val_w = np.array([0,0,120])
        max_val_w = np.array([255, 255, 255])
        #defining color thresholds
        '''min_val_y = np.array([50,120,130])
        max_val_y = np.array([20,230,230])
        min_val_w = np.array([0,0,150])
        max_val_w = np.array([240,240,240])'''


        roiy_end = frame.shape[0]
        roix_end = frame.shape[1]
        roi = img[self.road_horizon:roiy_end-20, 20:roix_end-20]
        #blur = cv2.medianBlur(roi, 5)
        blur = cv2.bilateralFilter(roi, 9, 21, 21)
        hsv = hsv_filter(blur, min_val_y, max_val_y,  min_val_w, max_val_w)
        cv2.imshow('hsv', hsv)
   

        
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        #roiy_end = frame.shape[0]
        #roix_end = frame.shape[1]
        roi_gray = gray[self.road_horizon:roiy_end-20, 20:roix_end-20]
        blur_gray = cv2.bilateralFilter(gray, 4, 17, 17)#cv2.medianBlur(roi, 11)
        th, thld = cv2.threshold(blur_gray, 130, 255, cv2.THRESH_BINARY)# binary threshold
        cv2.imshow('thld', thld)
        
        contours = cv2.Canny(blur, 60, 120)#canny edges
        cv2.imshow('canny', contours)
        

        
        if self.prob_hough:
            lines = cv2.HoughLinesP(contours, 1, np.pi/180, self.vote, minLineLength=30, maxLineGap=100)
        else:
            lines = self.standard_hough(contours, self.vote)

        if lines is not None:
            # find nearest lines to center
            lines = lines+np.array([0, self.road_horizon, 0, self.road_horizon]).reshape((1, 1, 4))  # scale points from ROI coordinates to full frame coordinates
            left_bound = None
            right_bound = None
            for l in lines:
                # find the rightmost line of the left half of the frame and the leftmost line of the right half
                for x1, y1, x2, y2 in l:
                    theta = np.abs(np.arctan2((y2-y1), (x2-x1)))  # line angle WRT horizon
                    if theta > self.roi_theta:  # ignore lines with a small angle WRT horizon
                        dist = self._base_distance(x1, y1, x2, y2, frame.shape[1])
                        if left_bound is None and dist < 0:
                            left_bound = (x1, y1, x2, y2)
                            left_dist = dist
                        elif right_bound is None and dist > 0:
                            right_bound = (x1, y1, x2, y2)
                            right_dist = dist
                        elif left_bound is not None and 0 > dist > left_dist:
                            left_bound = (x1, y1, x2, y2)
                            left_dist = dist
                        elif right_bound is not None and 0 < dist < right_dist:
                            right_bound = (x1, y1, x2, y2)
                            right_dist = dist
            if left_bound is not None:
                left_bound = self._scale_line(left_bound[0], left_bound[1], left_bound[2], left_bound[3], frame.shape[0])
            if right_bound is not None:
                right_bound = self._scale_line(right_bound[0], right_bound[1], right_bound[2], right_bound[3], frame.shape[0])

            return [left_bound, right_bound]


In [133]:
def gamma_correction_auto(RGBimage,equalizeHist = False): #0.35
    originalFile = RGBimage.copy()
    red = RGBimage[:,:,2]
    green = RGBimage[:,:,1]
    blue = RGBimage[:,:,0]
    
    forLuminance = cv2.cvtColor(originalFile,cv2.COLOR_BGR2YUV)
    Y = forLuminance[:,:,0]
    totalPix = vidsize[0]* vidsize[1]
    summ = np.sum(Y[:,:])
    Yaverage = np.divide(totalPix,summ)
    #Yclipped = np.clip(Yaverage,0,1)
    epsilon = 1.19209e-007
    correct_param = np.divide(-0.3,np.log10([Yaverage + epsilon]))
    correct_param = 0.7 - correct_param 

    red = red/255.0
    red = cv2.pow(red, correct_param)
    red = np.uint8(red*255)
    if equalizeHist:
        red = cv2.equalizeHist(red)
    
    green = green/255.0
    green = cv2.pow(green, correct_param)
    green = np.uint8(green*255)
    if equalizeHist:
        green = cv2.equalizeHist(green)
        
    
    blue = blue/255.0
    blue = cv2.pow(blue, correct_param)
    blue = np.uint8(blue*255)
    if equalizeHist:
        blue = cv2.equalizeHist(blue)
    

    output = cv2.merge((blue,green,red))
    #print(correct_param)
    return output

In [134]:
#capturing video
cap = cv2.VideoCapture('C:/Users/DarkLord/Downloads/giscle internship/projects/Lane detection_Giscle/input/videod.mp4')# it will capture videos from camara

vidsize = (640,480,3)

In [135]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    # Retrieve the number of color channels of the image.
    #channel_count = img.shape[2]
    
    # color used to fill polygon
    match_mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, (255,255,255))
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image

In [136]:



#defining corners for ROI
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)

h1 = int(height)
w1 = int(width)

topLeftPt = (width*(2/5), height*(3.0/5))
topRightPt = (width*(3/5), height*(3.0/5))

region_of_interest_points = [
(0, height),
#(0, height*(4.0/5)),
topLeftPt,
topRightPt,
#(width, height*(4.0/5)),
(width, height),
]

#defining color thresholds
min_val_y = np.array([50, 120, 130])
max_val_y = np.array([20, 230, 230])
min_val_w = np.array([150, 150, 150])
max_val_w = np.array([240, 240, 240])

ticks = 0

lt = LaneTracker(2, 0.1, 1500)
ld = LaneDetector(330)



# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('output/result.avi',fourcc, 20.0, (int(cap.get(3)),(int(cap.get(4)))))

while True:
    precTick = ticks
    ticks = cv2.getTickCount()
    dt = (ticks - precTick) / cv2.getTickFrequency()
    ret, frame = cap.read()
    if ret:
        #cv2.imshow('original',frame)
        #gamma = gamma_correction_auto(frame,equalizeHist = False) #0.2
        #cv2.imshow('gamma', gamma)
        #cropped = region_of_interest(frame, np.array([region_of_interest_points], np.int32))
        #cv2.imshow('cropped', cropped)
        #bilateral = cv2.bilateralFilter(cropped, 9, 80, 80)
        #cv2.imshow('bilateral', bilateral)
        #hsv = hsv_filter(cropped, min_val_y, max_val_y,  min_val_w, max_val_w)
        print(h1,w1)   
        predicted = lt.predict(dt)

        lanes = ld.detect(frame)
        
        if predicted is not None:
            cv2.line(frame, (predicted[0][0], predicted[0][1]), (predicted[0][2], predicted[0][3]), (0, 0, 255), 2)
            cv2.line(frame, (predicted[1][0], predicted[1][1]), (predicted[1][2], predicted[1][3]), (0, 0, 255), 2)
           
        lt.update(lanes)
        
        #cv2.imshow('hsv', hsv)
        #canny = cv2.Canny(hsv, 80, 255) #100
        #cv2.imshow('canny', canny)
        #hough, lines = hough_transform(frame, canny, 11, discard_horizontal = 0.7) #14 0.4
        #cv2.imshow('hough', hough)
        #_, frame = cap.read()
        #final = clustering(lines, frame, np.array([region_of_interest_points], np.int32), eps = 0.5, min_samples = 4)
        
       
        #out.write(frame)# write frames in the disk
        final = cv2.rectangle(frame,(20,315),(w1-20,h1),(0,255,0),2)
        cv2.imshow('final', final)
        

        
        k = cv2.waitKey(29) & 0xFF
        if k == ord('q'):
            break
            
cap.release()
#out.release()
cv2.destroyAllWindows()

480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
480 640
[(array([ 78.16000366], dtype=float32), array([ 480.], dtype=float32), array([ 270.16000366], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 78.16000366], dtype=float32), array([ 480.], dtype=float32), array([ 270.16000366], dtype=float32), array([ 330.], dtype=float32)), (array([ 331.], dtype=float32), array([ 330.], dtype=float32), array([ 553.05883789], dtype=float32), array([ 480.], dtype=float32))]
480 640
[(array([ 78.15997314], dtype=float32), array([ 480.], dtype=float32), array([ 270.16000366], 

480 640
[(array([ 76.5038147], dtype=float32), array([ 480.], dtype=float32), array([ 269.49789429], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 76.5038147], dtype=float32), array([ 480.], dtype=float32), array([ 269.49789429], dtype=float32), array([ 330.], dtype=float32)), (array([ 331.59399414], dtype=float32), array([ 329.88027954], dtype=float32), array([ 553.35681152], dtype=float32), array([ 479.82580566], dtype=float32))]
480 640
[(array([ 76.32868958], dtype=float32), array([ 480.], dtype=float32), array([ 269.57485962], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 76.32868958], dtype=float32), array([ 480.], dtype=float32), array([ 269.57485962], dtype=float32), array([ 330.], dtype=float32)), (array([ 331.7678833], dtype=float32), array([ 329.87466431], dtype=float32), array([ 553.46026611], dtype=float32), array([ 479.81765747], dtype=float32))]
480 640
[(array([ 76.14917755], dtype=float32), array([ 480.], dtype=float32), array([ 269.67965698], d

480 640
[(array([ 71.38114929], dtype=float32), array([ 480.], dtype=float32), array([ 275.79830933], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 71.38114929], dtype=float32), array([ 480.], dtype=float32), array([ 275.79830933], dtype=float32), array([ 330.], dtype=float32)), (array([ 324.52871704], dtype=float32), array([ 329.84249878], dtype=float32), array([ 528.56304932], dtype=float32), array([ 479.77096558], dtype=float32))]
480 640
[(array([ 71.03878021], dtype=float32), array([ 480.], dtype=float32), array([ 276.34030151], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 71.03878021], dtype=float32), array([ 480.], dtype=float32), array([ 276.34030151], dtype=float32), array([ 330.], dtype=float32)), (array([ 323.77563477], dtype=float32), array([ 329.84375], dtype=float32), array([ 525.66223145], dtype=float32), array([ 479.77276611], dtype=float32))]
480 640
[(array([ 70.67618561], dtype=float32), array([ 480.], dtype=float32), array([ 276.91415405], d

480 640
[(array([ 65.79238129], dtype=float32), array([ 480.], dtype=float32), array([ 286.06411743], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 65.79238129], dtype=float32), array([ 480.], dtype=float32), array([ 286.06411743], dtype=float32), array([ 330.], dtype=float32)), (array([ 215.64959717], dtype=float32), array([ 329.90682983], dtype=float32), array([ 392.89959717], dtype=float32), array([ 479.8644104], dtype=float32))]
480 640
[(array([ 65.61277008], dtype=float32), array([ 480.], dtype=float32), array([ 286.45245361], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 65.61277008], dtype=float32), array([ 480.], dtype=float32), array([ 286.45245361], dtype=float32), array([ 330.], dtype=float32)), (array([ 203.9523468], dtype=float32), array([ 329.91070557], dtype=float32), array([ 381.38568115], dtype=float32), array([ 479.87005615], dtype=float32))]
480 640
[(array([ 65.45561981], dtype=float32), array([ 480.], dtype=float32), array([ 286.8062439], d

480 640
[(array([ 63.58006668], dtype=float32), array([ 480.], dtype=float32), array([ 291.40106201], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 63.58006668], dtype=float32), array([ 480.], dtype=float32), array([ 291.40106201], dtype=float32), array([ 330.], dtype=float32)), (array([ 30.95310593], dtype=float32), array([ 329.97183228], dtype=float32), array([ 214.17710876], dtype=float32), array([ 479.95910645], dtype=float32))]
480 640
[(array([ 63.53785324], dtype=float32), array([ 480.], dtype=float32), array([ 291.54156494], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 63.53785324], dtype=float32), array([ 480.], dtype=float32), array([ 291.54156494], dtype=float32), array([ 330.], dtype=float32)), (array([ 24.59142876], dtype=float32), array([ 329.97427368], dtype=float32), array([ 208.18899536], dtype=float32), array([ 479.96264648], dtype=float32))]
480 640
[(array([ 63.48341751], dtype=float32), array([ 480.], dtype=float32), array([ 291.70126343], 

...

480 640
[(array([ 7.05509567], dtype=float32), array([ 480.], dtype=float32), array([ 199.9941864], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 7.05509567], dtype=float32), array([ 480.], dtype=float32), array([ 199.9941864], dtype=float32), array([ 330.], dtype=float32)), (array([-36.78071213], dtype=float32), array([ 330.01705933], dtype=float32), array([ 127.77314758], dtype=float32), array([ 480.02490234], dtype=float32))]
480 640
[(array([ 2.6642735], dtype=float32), array([ 480.], dtype=float32), array([ 193.05105591], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 2.6642735], dtype=float32), array([ 480.], dtype=float32), array([ 193.05105591], dtype=float32), array([ 330.], dtype=float32)), (array([-35.59788513], dtype=float32), array([ 330.01687622], dtype=float32), array([ 128.63768005], dtype=float32), array([ 480.02462769], dtype=float32))]
480 640
[(array([-1.67230511], dtype=float32), array([ 480.], dtype=float32), array([ 186.19740295], dtype=flo

480 640
[(array([-58.95468903], dtype=float32), array([ 480.], dtype=float32), array([ 95.32034302], dtype=float32), array([ 330.], dtype=float32))]
[(array([-58.95468903], dtype=float32), array([ 480.], dtype=float32), array([ 95.32034302], dtype=float32), array([ 330.], dtype=float32)), (array([-14.87734604], dtype=float32), array([ 330.01248169], dtype=float32), array([ 150.05625916], dtype=float32), array([ 480.01803589], dtype=float32))]
480 640
[(array([-61.12979126], dtype=float32), array([ 480.], dtype=float32), array([ 91.85197449], dtype=float32), array([ 330.], dtype=float32))]
[(array([-61.12979126], dtype=float32), array([ 480.], dtype=float32), array([ 91.85197449], dtype=float32), array([ 330.], dtype=float32)), (array([-13.93053436], dtype=float32), array([ 330.01220703], dtype=float32), array([ 151.44061279], dtype=float32), array([ 480.01763916], dtype=float32))]
480 640
[(array([-63.12424088], dtype=float32), array([ 480.], dtype=float32), array([ 88.66600037], dtype

480 640
[(array([-42.65433121], dtype=float32), array([ 480.], dtype=float32), array([ 88.62450409], dtype=float32), array([ 330.], dtype=float32))]
[(array([-42.65433121], dtype=float32), array([ 480.], dtype=float32), array([ 88.62450409], dtype=float32), array([ 330.], dtype=float32)), (array([ 1.78414011], dtype=float32), array([ 330.0071106], dtype=float32), array([ 174.39857483], dtype=float32), array([ 480.01025391], dtype=float32))]
480 640
[(array([-33.60231781], dtype=float32), array([ 480.], dtype=float32), array([ 96.0486145], dtype=float32), array([ 330.], dtype=float32))]
[(array([-33.60231781], dtype=float32), array([ 480.], dtype=float32), array([ 96.0486145], dtype=float32), array([ 330.], dtype=float32)), (array([ 2.47489977], dtype=float32), array([ 330.00689697], dtype=float32), array([ 175.40861511], dtype=float32), array([ 480.00997925], dtype=float32))]
480 640
[(array([-24.31315041], dtype=float32), array([ 480.], dtype=float32), array([ 103.93292236], dtype=flo

480 640
[(array([ 94.62171936], dtype=float32), array([ 480.], dtype=float32), array([ 221.71401978], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 94.62171936], dtype=float32), array([ 480.], dtype=float32), array([ 221.71401978], dtype=float32), array([ 330.], dtype=float32)), (array([ 12.57143879], dtype=float32), array([ 330.00296021], dtype=float32), array([ 190.13272095], dtype=float32), array([ 480.00430298], dtype=float32))]
480 640
[(array([ 99.43047333], dtype=float32), array([ 480.], dtype=float32), array([ 226.89367676], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 99.43047333], dtype=float32), array([ 480.], dtype=float32), array([ 226.89367676], dtype=float32), array([ 330.], dtype=float32)), (array([ 27.36925888], dtype=float32), array([ 330.0027771], dtype=float32), array([ 207.3087616], dtype=float32), array([ 480.00405884], dtype=float32))]
480 640
[(array([ 103.31044006], dtype=float32), array([ 480.], dtype=float32), array([ 231.68885803], d

480 640
[(array([ 146.06089783], dtype=float32), array([ 480.], dtype=float32), array([ 296.71209717], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 146.06089783], dtype=float32), array([ 480.], dtype=float32), array([ 296.71209717], dtype=float32), array([ 330.], dtype=float32)), (array([ 237.13389587], dtype=float32), array([ 330.00048828], dtype=float32), array([ 430.7545166], dtype=float32), array([ 480.00085449], dtype=float32))]
480 640
[(array([ 147.17062378], dtype=float32), array([ 480.], dtype=float32), array([ 299.13659668], dtype=float32), array([ 330.], dtype=float32))]
[(array([ 147.17062378], dtype=float32), array([ 480.], dtype=float32), array([ 299.13659668], dtype=float32), array([ 330.], dtype=float32)), (array([ 244.93132019], dtype=float32), array([ 330.00039673], dtype=float32), array([ 438.42681885], dtype=float32), array([ 480.00073242], dtype=float32))]
480 640
[(array([ 148.29457092], dtype=float32), array([ 480.], dtype=float32), array([ 301.54501